In [2]:
!pip install vecstack

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19864 sha256=4d7ca9b181f760b39e3bb0db556f2ba95888dde8e903fd50ed37ce32b861dc73
  Stored in directory: c:\users\dipti\appdata\local\pip\cache\wheels\7e\ee\d6\47cb94a403bc544de1433986e5530d6b0498021098fbe43aa1
Successfully built vecstack


In [4]:
!pip install imblearn

     -------------------------------------- 199.3/199.3 kB 6.1 MB/s eta 0:00:00


In [5]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from collections import Counter #for Smote, 

import warnings
warnings.filterwarnings("ignore")

In [6]:
trainfile = r'C:\Users\dipti\Documents\CIS508\Assignment 3\RevisedHomesiteTrain1.csv'
train_data = pd.read_csv(trainfile)

#train_data = pd.read_csv("C:\Users\dipti\Documents\CIS508\Assignment 3\RevisedHomesiteTrain1.csv")


testfile = r'C:\Users\dipti\Documents\CIS508\Assignment 3\RevisedHomesiteTest1.csv'
test_data = pd.read_csv(testfile)

#test_data = pd.read_csv("C:\Users\dipti\Documents\CIS508\Assignment 3\RevisedHomesiteTest1.csv")


print(train_data.shape)
print(test_data.shape)
print(train_data.head())    

(65000, 596)
(173836, 596)
   CoverageField11A  CoverageField11B  CoverageField1A  CoverageField1B  \
0                 2                 1               17               23   
1                 5                 9                6                8   
2                 4                 6                7               12   
3                15                23                3                2   
4                 4                 6                8               13   

   CoverageField2A  CoverageField2B  CoverageField3A  CoverageField3B  \
0               17               23               15               22   
1                6                8                5                7   
2                7               12                6               10   
3                3                2                2                2   
4                8               13                7               11   

   CoverageField4A  CoverageField4B  ...  PropertyField38_N  \
0               16  

In [10]:
X_train=train_data.iloc[:, :-1]
X_test=test_data.iloc[:, :-1]
y_train=train_data.iloc[:,-1]
y_test=test_data.iloc[:,-1]

print(y_train.shape)
print(y_test.shape)

(65000,)
(173836,)


# SMOTE

In [12]:
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy=0.5)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 26369})


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size= 0.3, random_state=0)

In [15]:
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY 
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)
dt_predict=dt.predict(X_test)
print("accuracy Score (training) for Decision Tree:{0:6f}".format(dt.score(X_test,y_test)))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(y_test, dt_predict))

accuracy Score (training) for Decision Tree:0.880359
Confusion Matrix for Decision Tree
[[14631  1166]
 [ 1167  2536]]


In [17]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
dt_random = RandomizedSearchCV(dt, parameters, n_iter=15)
dt_random.fit(X_train, y_train)
grid_parm=dt_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
dt = DecisionTreeClassifier(**grid_parm)
dt.fit(X_train, y_train)
dt_predict = dt.predict(X_test)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning for Decision Tree:{0:6f}".format(dt.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,dt_predict))
print("=== Classification Report ===")
print(classification_report(y_test,dt_predict))

#get cross-validation report
dt_cv_score = cross_val_score(df, X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(dt_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",dt_cv_score.mean())

{'min_samples_split': 10, 'max_depth': 9}
accuracy Score (training) after hypertuning for Decision Tree:0.911590
Confusion Matrix after hypertuning for Decision Tree
[[15549   248]
 [ 1476  2227]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.91      0.98      0.95     15797
           1       0.90      0.60      0.72      3703

    accuracy                           0.91     19500
   macro avg       0.91      0.79      0.83     19500
weighted avg       0.91      0.91      0.90     19500

=== All AUC Scores ===
[0.816992   0.81590189 0.81589462 0.81211196 0.81141432 0.82028953
 0.81536047 0.80866453 0.81994071 0.80781204]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.8144382080308523


In [18]:
#Construct Random Forest Model

rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_predict=rfc.predict(X_test)
print("accuracy Score (training) for RandomForest:{0:6f}".format(rfc.score(X_test,y_test)))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))

accuracy Score (training) for RandomForest:0.903385
Confusion Matrix for Random Forest:
[[15590   207]
 [ 1677  2026]]


In [19]:
#Hyperparameter tuning for random forest classifier
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(X_train, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)

#Construct Random Forest with best parameters
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(X_train, y_train)
rfc_predict = rfc.predict(X_test)
print("accuracy Score (training) after hypertuning for Random Forest:{0:6f}".format(rfc.score(X_test, y_test)))
print("Confusion Matrix after hypertuning for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,rfc_predict))

#get cross-validation report
rfc_cv_score = cross_val_score(rfc, X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",rfc_cv_score.mean())


{'min_samples_split': 50, 'max_depth': 19}
accuracy Score (training) after hypertuning for Random Forest:0.902718
Confusion Matrix after hypertuning for Random Forest:
[[15610   187]
 [ 1710  1993]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.90      0.99      0.94     15797
           1       0.91      0.54      0.68      3703

    accuracy                           0.90     19500
   macro avg       0.91      0.76      0.81     19500
weighted avg       0.90      0.90      0.89     19500

=== All AUC Scores ===
[0.95342125 0.94774015 0.94034593 0.94747134 0.9431947  0.94832299
 0.94876068 0.94444989 0.94681196 0.94620263]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.9466721505087559


In [20]:
#Construct MultiLayer Perceptron Model
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(max_iter=100)
mlp.fit(X_train, y_train)
mlp_predict=mlp.predict(X_test)
print("accuracy Score (training) for MultiLayer Perceptron:{0:6f}".format(mlp.score(X_test,y_test)))
print("Confusion Matrix for MultiLayer Perceptron:")
print(confusion_matrix(y_test,mlp_predict))

accuracy Score (training) for MultiLayer Perceptron:0.761385
Confusion Matrix for MultiLayer Perceptron:
[[11865  3932]
 [  721  2982]]


In [22]:
#Hyperparameter tuning done for MultiLayer Perceptron classifier

#parameters = {'hidden_layer_sizes':[(10,), (20,)], 'activation':['tanh', 'relu'], 'solver':['sgd', 'adam'], 'alpha': [0.0001, 0.05], 'learning_rate':['constant', 'adaptive']}
#parameters = {'hidden_layer_sizes':[(10,5), (20,5)], 'activation':['tanh', 'relu'], 'learning_rate':['constant', 'adaptive']}
parameters = {'hidden_layer_sizes':[(10,5,3), (20,7,3)], 'activation':['tanh', 'relu'], 'learning_rate':['constant', 'adaptive'], 'max_iter' :[100, 150]}
#parameters = {'hidden_layer_sizes':[(10,), (15,), (10,5), (20,7,3)]}

mlp_random = RandomizedSearchCV(mlp,parameters,n_iter=15)
mlp_random.fit(X_train, y_train)
grid_parm=mlp_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the MLPClassifier 
mlp = MLPClassifier(**grid_parm)
mlp.fit(X_train, y_train)
mlp_predict = mlp.predict(X_test)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning for MultiLayer Perceptron:{0:6f}".format(mlp.score(X_test,  y_test)))
print("Confusion Matrix after hypertuning for MultiLayer Perceptron")
print(confusion_matrix(y_test, mlp_predict))
print("=== Classification Report ===")
print(classification_report(y_test, mlp_predict))

#get cross-validation report
mlp_cv_score = cross_val_score(mlp, X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(mlp_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - MultiLayer Perceptron: ",mlp_cv_score.mean())


{'max_iter': 100, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (10, 5, 3), 'activation': 'relu'}
accuracy Score (training) after hypertuning for MultiLayer Perceptron:0.810103
Confusion Matrix after hypertuning for MultiLayer Perceptron
[[15797     0]
 [ 3703     0]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.81      1.00      0.90     15797
           1       0.00      0.00      0.00      3703

    accuracy                           0.81     19500
   macro avg       0.41      0.50      0.45     19500
weighted avg       0.66      0.81      0.73     19500

=== All AUC Scores ===
[0.5        0.50054142 0.50094748 0.5        0.87384126 0.52298673
 0.5        0.5        0.5        0.85389518]


=== Mean AUC Score ===
Mean AUC Score - MultiLayer Perceptron:  0.5752212070475483


In [23]:
#Construct K-Nearest Neighbor Model
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)

neigh.fit(X_train, y_train)
neigh_predict=neigh.predict(X_test)
print("accuracy Score (training) for KNeighborsClassifier:{0:6f}".format(neigh.score(X_test, y_test)))
print("Confusion Matrix for KNeighborsClassifier:")
print(confusion_matrix(y_test, neigh_predict))

accuracy Score (training) for KNeighborsClassifier:0.753692
Confusion Matrix for KNeighborsClassifier:
[[14292  1505]
 [ 3298   405]]


In [24]:
#Hyperparameter tuning done for K-Nearest Neighbor classifier

parameters = {'n_neighbors':[3,5,7,9,11], 'weights':['uniform', 'distance'], 'p':[1,2]}


neigh_random = RandomizedSearchCV(neigh,parameters,n_iter=15)
neigh_random.fit(X_train, y_train)
grid_parm=neigh_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the MLPClassifier 
neigh = KNeighborsClassifier(**grid_parm)
neigh.fit(X_train,y_train)
neigh_predict = neigh.predict(X_test)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning for KNeighborsClassifier:{0:6f}".format(neigh.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for KNeighborsClassifier")
print(confusion_matrix(y_test,neigh_predict))
print("=== Classification Report ===")
print(classification_report(y_test,neigh_predict))

#get cross-validation report
neigh_cv_score = cross_val_score(neigh, X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(neigh_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - KNeighborsClassifier: ",neigh_cv_score.mean())


{'weights': 'uniform', 'p': 1, 'n_neighbors': 11}
accuracy Score (training) after hypertuning for KNeighborsClassifier:0.803385
Confusion Matrix after hypertuning for KNeighborsClassifier
[[15569   228]
 [ 3606    97]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.81      0.99      0.89     15797
           1       0.30      0.03      0.05      3703

    accuracy                           0.80     19500
   macro avg       0.56      0.51      0.47     19500
weighted avg       0.71      0.80      0.73     19500

=== All AUC Scores ===
[0.56505276 0.58231317 0.5636328  0.56978559 0.56973942 0.56409089
 0.57719863 0.56482285 0.56666785 0.5677777 ]


=== Mean AUC Score ===
Mean AUC Score - KNeighborsClassifier:  0.5691081654541607


In [25]:
#Construct Linear Support Vector Machine Model
from sklearn.svm import LinearSVC 
linsvm = LinearSVC(max_iter=300) 
linsvm.fit(X_train, y_train) 
linsvm_predict=linsvm.predict(X_test) 
print("accuracy Score (training) for Linear SVM Classifier:{0:6f}".format(linsvm.score(X_test,y_test))) 
print("Confusion Matrix for Linear SVM Classifier:") 
print(confusion_matrix(y_test,linsvm_predict))

accuracy Score (training) for Linear SVM Classifier:0.760564
Confusion Matrix for Linear SVM Classifier:
[[14247  1550]
 [ 3119   584]]


In [28]:
#Construct Support Vector Machine Model
from sklearn.svm import SVC 
svm = SVC(max_iter=500) 
svm.fit(X_train, y_train) 
svm_predict=svm.predict(X_test) 
print("accuracy Score (training) for SVM Classifier:{0:6f}".format(svm.score(X_test,y_test))) 
print("Confusion Matrix for SVM Classifier:") 
print(confusion_matrix(y_test,svm_predict))

accuracy Score (training) for SVM Classifier:0.394103
Confusion Matrix for SVM Classifier:
[[ 5231 10566]
 [ 1249  2454]]


In [29]:
#Construct Gradient Boosting model

search_grid={'n_estimators':[5,10,20],'learning_rate':[0.01,.1]}
abc =GradientBoostingClassifier()
abc.fit(X_train, y_train)
abc_predict=abc.predict(X_test)
print("accuracy Score (training) for Boosting:{0:6f}".format(abc.score(X_test,y_test)))
print("Confusion Matrix for boosting:")
print(confusion_matrix(y_test,abc_predict))
abc_random = RandomizedSearchCV(abc,search_grid,n_iter=15)
abc_random.fit(X_train, y_train)
grid_parm_abc=abc_random.best_params_
print(grid_parm_abc)
abc= GradientBoostingClassifier(**grid_parm_abc)
abc.fit(X_train,y_train)
abc_predict = abc.predict(X_test)
print("accuracy Score (training) after hypertuning for Boosting:{0:6f}".format(abc.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Boosting:")
print(confusion_matrix(y_test,abc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,abc_predict))
abc_cv_score = cross_val_score(abc, X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(abc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Boosting: ",abc_cv_score.mean())

accuracy Score (training) for Boosting:0.920308
Confusion Matrix for boosting:
[[15622   175]
 [ 1379  2324]]
{'n_estimators': 20, 'learning_rate': 0.1}
accuracy Score (training) after hypertuning for Boosting:0.895949
Confusion Matrix after hypertuning for Boosting:
[[15783    14]
 [ 2015  1688]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.89      1.00      0.94     15797
           1       0.99      0.46      0.62      3703

    accuracy                           0.90     19500
   macro avg       0.94      0.73      0.78     19500
weighted avg       0.91      0.90      0.88     19500

=== All AUC Scores ===
[0.94134938 0.93192231 0.93499847 0.94270752 0.93088929 0.93749668
 0.93822168 0.93283042 0.9329906  0.9379878 ]


=== Mean AUC Score ===
Mean AUC Score - Boosting:  0.9361394155006826


In [30]:
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ KNeighborsClassifier(), MLPClassifier(), SVC(), LinearSVC(), RandomForestClassifier(), DecisionTreeClassifier() ]
      
S_Train, S_Test = stacking(models,                   
                           X_res, y_res, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)


___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [6]

model  0:     [KNeighborsClassifier]
    fold  0:  [0.69899378]
    fold  1:  [0.69676897]
    fold  2:  [0.70121859]
    fold  3:  [0.70145631]
    ----
    MEAN:     [0.69960941] + [0.00190050]
    FULL:     [0.69960939]

model  1:     [MLPClassifier]
    fold  0:  [0.87647267]
    fold  1:  [0.79900895]
    fold  2:  [0.86934318]
    fold  3:  [0.85998180]
    ----
    MEAN:     [0.85120165] + [0.03069572]
    FULL:     [0.85120154]

model  2:     [SVC]
    fold  0:  [0.66668352]
    fold  1:  [0.66668352]
    fold  2:  [0.66663296]
    fold  3:  [0.66666667]
    ----
    MEAN:     [0.66666667] + [0.00002064]
    FULL:     [0.66666667]

model  3:     [LinearSVC]
    fold  

In [31]:
model = MLPClassifier()
    
model = model.fit(S_Train, y_res)
y_pred = model.predict(S_Test)
print('Final prediction score for ensemble methods: [%.8f]' % accuracy_score(y_test, y_pred))

Final prediction score for ensemble methods: [1.00000000]


In [32]:
model = KNeighborsClassifier()
    
model = model.fit(S_Train, y_res)
y_pred = model.predict(S_Test)
print('Final prediction score for ensemble methods: [%.8f]' % accuracy_score(y_test, y_pred))

Final prediction score for ensemble methods: [0.99984615]


In [33]:
model = GradientBoostingClassifier()
    
model = model.fit(S_Train, y_res)
y_pred = model.predict(S_Test)
print('Final prediction score for ensemble methods: [%.8f]' % accuracy_score(y_test, y_pred))

Final prediction score for ensemble methods: [1.00000000]


In [34]:
#Get Prediction Probability for the predicted class as a dataframe
pred_Probability =pd.DataFrame(model.predict_proba(S_Test))

pred_Probability.head()

,0,1
0,0.965668,0.034332
1,0.965668,0.034332
2,0.965668,0.034332
3,0.965668,0.034332
4,0.965668,0.034332


In [41]:
# Create submission for Kaggle
sub= pd.DataFrame()
sub ["QuoteNumber"]= test_data["QuoteNumber"]
sub ["QuoteConversion_Flag"] = y_pred

sub.to_csv('assgn3.csv', index= False)

ValueError: Length of values (19500) does not match length of index (173836)